In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)
from selenium import webdriver

from selenium.webdriver.common.by import By

import urllib.request
import urllib.parse
import time
import re #정규식

In [5]:
url = 'https://everytime.kr/timetable'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)
user_id = input('에브리타임 아이디를 입력해주세요.: ')
et_login = driver.find_element(By.NAME, "id")
et_login.clear()
et_login.send_keys(user_id)

user_pw = input('에브리타임 비밀번호를 입력해주세요.: ')
et_login = driver.find_element(By.NAME, "password")
et_login.clear()
et_login.send_keys(user_pw) #자신의 비번을 넣으세요

driver.find_element(By.XPATH, '//input[@type="submit" and @value="에브리타임 로그인"]').click()
time.sleep(3)
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="sheet"]/ul/li[3]/a""").click()
        time.sleep(2)
        break
    except:
        continue
timetable_all = pd.DataFrame()
timetable_all
graph = {}
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="container"]/ul/li[1]""").click()
        time.sleep(2)
        break
    except:
        continue
for i in range(9):
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="subjects"]/div[1]/a[4]""").click()
            time.sleep(2)
            break
        except:
            continue
    while i == 0:
        try:
            driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/li[2]""").click()
            time.sleep(2)
            break
        except:
            continue
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/li["""+str((i+1))+"""]""").click()
            time.sleep(2)
            break
        except:
            continue
#     while True:
#         try:
#             driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/ul["""+str((i+1))+"""]/li""").click()
#             time.sleep(2)
#             break
#         except:
#             continue
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    contents = []
    tmp = soup.find('div','list').find('tbody').find_all('td')
    for i in tmp:
        tmp = i.get_text()
        contents.append(tmp)
    contents = np.array(contents)
    column = []
    tmp = tmp = soup.find('div','list').find('thead').find_all('th')
    for i in tmp:
        tmp = str(i).lstrip('<th>').split('<div>')
        column.append(tmp[0])
    timetable = pd.DataFrame(contents.reshape(len(contents)//12,12), columns=column)
    del(timetable['계획서']) ;del(timetable['강의평'])
    major = soup.find('a','item active').get_text().split(':')[1]
    graph[str(major)] = len(timetable)
    timetable_all = pd.concat([timetable_all,timetable],ignore_index=True)
    timetable.to_excel('../data/19년 1학기 '+major+' 강좌.xls',encoding = 'utf8',sheet_name=major,index = False) # 학과 별로 저장
timetable_all.to_excel('../data/19년 1학기 공과대학 강좌.xls',encoding = 'utf8',sheet_name=major,index = False) # 공대 전체 데이터 저장
time.sleep(2)
driver.close()

KeyboardInterrupt: 

In [3]:
driver = webdriver.Chrome()

# 에브리타임 로그인 페이지 접속
url = 'https://everytime.kr/timetable'
driver.get(url)
time.sleep(3)

# 사용자 입력 받기
user_id = input('에브리타임 아이디를 입력해주세요.: ')
et_login = driver.find_element(By.NAME, "id")
et_login.clear()
et_login.send_keys(user_id)

user_pw = input('에브리타임 비밀번호를 입력해주세요.: ')
et_pw = driver.find_element(By.NAME, "password")
et_pw.clear()
et_pw.send_keys(user_pw)

# 로그인 버튼 클릭
driver.find_element(By.XPATH, '//input[@type="submit" and @value="에브리타임 로그인"]').click()
time.sleep(3)

# 특정 시트로 이동
while True:
    try:
        driver.find_element(By.XPATH, """//*[@id="sheet"]/ul/li[3]/a""").click() #<li class="button search">수업 목록에서 검색</li>
        time.sleep(2)
        break
    except:
        continue

timetable_all = pd.DataFrame()
graph = {}

# 데이터 추출을 위한 반복
for i in range(9):
    while True:
        try:
            driver.find_element(By.XPATH, """//*[@id="subjects"]/div[1]/a[4]""").click() #<a class="item" data-id="category"><span class="key">전공/영역:</span><span class="value">전체</span><span class="reset"></span></a>
            time.sleep(2)
            break
        except:
            continue

    if i == 0:
        while True:
            try:
                driver.find_element(By.XPATH, """//*[@id="subjectCategoryFilter"]/div/ul/li[2]""").click()
                time.sleep(2)
                break
            except:
                continue

    while True:
        try:
            driver.find_element(By.XPATH, f"""//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/li[{i+1}]""").click()
            time.sleep(2)
            break
        except:
            continue

    # 페이지 소스 가져오기 및 파싱
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    contents = []

    # 데이터 추출
    tmp = soup.find('div', class_='list').find('tbody').find_all('td')
    for td in tmp:
        contents.append(td.get_text())

    contents = np.array(contents)

    # 칼럼 정보 추출
    column = []
    headers = soup.find('div', class_='list').find('thead').find_all('th')
    for th in headers:
        column.append(th.get_text().strip())

    # DataFrame 생성
    timetable = pd.DataFrame(contents.reshape(len(contents)//12, 12), columns=column)
    timetable.drop(columns=['계획서', '강의평'], inplace=True)

    # 학과 정보 추출
    major = soup.find('a', class_='item active').get_text().split(':')[1]
    graph[major] = len(timetable)

    # DataFrame 합치기
    timetable_all = pd.concat([timetable_all, timetable], ignore_index=True)
    timetable.to_excel(f'../data/19년 1학기 {major} 강좌.xls', encoding='utf8', sheet_name=major, index=False)

# 전체 데이터 저장
timetable_all.to_excel('../data/19년 1학기 공과대학 강좌.xls', encoding='utf8', sheet_name='공과대학', index=False)

# WebDriver 종료
time.sleep(2)
driver.close()

In [43]:
#-*- coding:utf-8 -*-
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from openpyxl import Workbook,load_workbook

# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
driver = webdriver.Chrome()

# url에 접근한다.
driver.get('https://everytime.kr/timetable')
# 암묵적으로 웹 자원 로드를 위해 5초까지 기다려 준다.
driver.implicitly_wait(5)

# 아이디/비밀번호를 입력해준다.
driver.find_element(By.NAME,'id').send_keys('gnbupi')
driver.find_element(By.NAME,'password').send_keys('rlatkddn1212')
sleep(2)

# 로그인 버튼을 눌러주자.
driver.find_element(By.XPATH, '//input[@type="submit" and @value="에브리타임 로그인"]').click()


#수업 목록에서 검색 클릭
driver.find_element(By.XPATH,'//li[@class="button search"]').click()
driver.find_element(By.CSS_SELECTOR, 'span.key').click()
#팝업창 닫기
sleep(2)
#driver.find_element_by_xpath('//*[@id="sheet"]/ul/li[3]/a').click()


sleep(2)

def expand_all_categories():
    
    while True:
        # 모든 "parent" 항목 찾기
        parent_elements = driver.find_elements(By.CSS_SELECTOR, 'li.parent')
        expanded_any = False
        for parent in parent_elements:
            if 'unfolded' not in parent.get_attribute('class'):
                parent.click()
                sleep(1)  # 페이지 업데이트 대기
                expanded_any = True
        if not expanded_any:
            break



results = []
count=0


name=[]
name.append('id')

name.append('department')

name.append('grade')
name.append('course_type')
name.append('course_number')
name.append('course_name')

name.append('credits')
name.append('capacity')



name.append('professor')
name.append('class_time')
name.append('class_place')
name.append('class_type')

results.append(name)

# 모든 <li class="child"> 요소 클릭
def click_all_children():
    while True:
        # 모든 "child" 항목 찾기
        child_elements = driver.find_elements(By.CSS_SELECTOR, 'li.child')
        if not child_elements:
            break
        for child in child_elements:

            
            expand_all_categories()
            child.click()
            sleep(2)  # 페이지 업데이트 대기
            # 다시 "child" 항목을 로드하기 위해 스크롤 할 수도 있습니다
            driver.execute_script('arguments[0].scrollIntoView(true);', child)
            sleep(1)
            pre_count = 0
            #스크롤 맨아래로 내리기
            while True:
                #tr요소 접근
                element = driver.find_elements(By.CSS_SELECTOR, "#subjects > div.list > table > tbody > tr")

                # tr 마지막 요소 접근
                result = element[-1]
                #마지막요소에 focus주기
                driver.execute_script('arguments[0].scrollIntoView(true);',result)
                sleep(2)

                #현재 접근한 요소의 갯수
                current_count = len(element)
                if pre_count == current_count:
                    break
                #같지않다면
                pre_count = current_count


            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            trs = soup.select('#subjects > div.list > table > tbody > tr')








            for tr in trs:
                global count
                result=[]
                tds = tr.select('#subjects > div.list > table > tbody > tr > td')
                
                count=count+1
                result.append(count) #id

                result.append(child) #학과
                
                result.append(tds[6].text) #학년
                result.append(tds[5].text) #구분
                result.append(tds[0].text) #과목코드
                result.append(tds[1].text) #과목명

                result.append(tds[7].text) #학점
                result.append(tds[10].text) #정원
            
                

                result.append(tds[2].text) #교수
                result.append(tds[3].text) #강의시간
                result.append(tds[4].text) #강의실
                result.append(tds[12].text) #비고
                
                
                results.append(result)





click_all_children()










#값이 들어있다면!
if results:
    print("성공!!")
excel_column = 9
write_wb = Workbook()
write_ws = write_wb.create_sheet('result.xls')
for data in results:
    write_ws = write_wb.active
    write_ws.append(data)
write_wb.save('C:/Users/gnbup/Desktop/code/deeplearning/result.csv')

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=127.0.6533.122)
Stacktrace:
	GetHandleVerifier [0x00007FF7678A9642+30946]
	(No symbol) [0x00007FF76785E3D9]
	(No symbol) [0x00007FF767756E09]
	(No symbol) [0x00007FF7677A9712]
	(No symbol) [0x00007FF76779C271]
	(No symbol) [0x00007FF7677CCA6A]
	(No symbol) [0x00007FF76779BBB6]
	(No symbol) [0x00007FF7677CCC80]
	(No symbol) [0x00007FF7677EB041]
	(No symbol) [0x00007FF7677CC813]
	(No symbol) [0x00007FF76779A6E5]
	(No symbol) [0x00007FF76779B021]
	GetHandleVerifier [0x00007FF7679DF84D+1301229]
	GetHandleVerifier [0x00007FF7679EBDC7+1351783]
	GetHandleVerifier [0x00007FF7679E2A13+1313971]
	GetHandleVerifier [0x00007FF7678DDD16+245686]
	(No symbol) [0x00007FF76786759F]
	(No symbol) [0x00007FF767863814]
	(No symbol) [0x00007FF7678639A2]
	(No symbol) [0x00007FF76785A3FF]
	BaseThreadInitThunk [0x00007FFA66B2257D+29]
	RtlUserThreadStart [0x00007FFA67F2AF28+40]


In [76]:
# -*- coding:utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep
from openpyxl import Workbook

# 웹 드라이버 설정
driver = webdriver.Chrome()

# 웹 페이지 열기
driver.get('https://everytime.kr/timetable')
driver.implicitly_wait(5)

# 로그인
driver.find_element(By.NAME, 'id').send_keys('gnbupi')
driver.find_element(By.NAME, 'password').send_keys('rlatkddn1212')
sleep(2)
driver.find_element(By.XPATH, '//input[@type="submit" and @value="에브리타임 로그인"]').click()

# 수업 목록에서 검색 클릭
driver.find_element(By.XPATH, '//li[@class="button search"]').click()

# 팝업창 닫기
sleep(2)


driver.find_element(By.CSS_SELECTOR, 'span.key').click()
sleep(1)



# 모든 카테고리 확장
def expand_all_categories():
    
    while True:
        parent_elements = driver.find_elements(By.CSS_SELECTOR, 'li.parent')
        expanded_any = False
        for parent in parent_elements:
            if 'unfolded' not in parent.get_attribute('class'):
                parent.click()
                sleep(1)
                expanded_any = True
        if not expanded_any:
            break

results = []
count = 0

# 엑셀 열 헤더 추가
name = [
    'id', 'department', 'grade', 'course_type', 'course_number', 'course_name',
    'credits', 'capacity', 'professor', 'class_time', 'class_place', 'class_type'
]
results.append(name)

# 모든 <li class="child"> 요소 클릭
def click_all_children():
    
    child_elements = driver.find_elements(By.CSS_SELECTOR, 'li.child')
    for i in range(len(child_elements)):
        # expand_all_categories 함수를 호출하여 페이지를 업데이트하기 전에
        # 매번 child_elements 리스트를 새로 가져옵니다.

        

        expand_all_categories()
        child_elements = driver.find_elements(By.CSS_SELECTOR, 'li.child')
        child = child_elements[i]
        try:
            # <a> 태그가 있는 경우
            department_name = child.find_element(By.CSS_SELECTOR, 'a').text
        except:
            # <a> 태그가 없는 경우, 직접 <li> 요소의 텍스트 사용
            department_name = child.text
        
        print(f"현제 학과: {department_name}")

        driver.execute_script('arguments[0].scrollIntoView(true);', child)
        sleep(1)  # 페이지 업데이트 대기
        child.click()
        sleep(2)
        pre_count = 0

        # 스크롤 맨 아래로 내리기
        while True:
            element = driver.find_elements(By.CSS_SELECTOR, "#subjects > div.list > table > tbody > tr")
            result = element[-1]
            driver.execute_script('arguments[0].scrollIntoView(true);', result)
            sleep(2)
            current_count = len(element)
            if pre_count == current_count:
                break
            pre_count = current_count

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        trs = soup.select('#subjects > div.list > table > tbody > tr')

        for tr in trs:
            global count
            result = []
            tds = tr.find_all('td')
            
            count += 1
            result.append(count)  # id

            result.append(department_name)  # 학과
            
            if len(tds) > 12:
                result.append(tds[6].text)  # 학년
                result.append(tds[5].text)  # 구분
                result.append(tds[0].text)  # 과목코드
                result.append(tds[1].text)  # 과목명
                result.append(tds[7].text)  # 학점
                result.append(tds[10].text)  # 정원
                result.append(tds[2].text)  # 교수
                result.append(tds[3].text)  # 강의시간
                result.append(tds[4].text)  # 강의실
                result.append(tds[12].text)  # 비고
            
            results.append(result)

        # 다음 child 요소로 이동하기 전에 다시 'span.key'를 클릭합니다.
        driver.find_element(By.CSS_SELECTOR, 'span.key').click()
        sleep(1)

click_all_children()

# 엑셀 파일 저장
if results:
    print("성공!!")
    write_wb = Workbook()
    write_ws = write_wb.active
    for data in results:
        write_ws.append(data)
    write_wb.save('C:/Users/gnbup/Desktop/code/deeplearning/result.csv')

driver.quit()


현제 학과: 기초교양
현제 학과: 핵심1영역
현제 학과: 핵심2영역
현제 학과: 핵심3영역
현제 학과: 핵심4영역
현제 학과: 핵심4-1영역(이공계핵심기초)
현제 학과: 일반교양
현제 학과: 교직
현제 학과: 자유선택
현제 학과: 컴퓨터소프트웨어특성화대학
현제 학과: 컴퓨터정보공학과
현제 학과: 인공지능융합학과
현제 학과: 임베디드소프트웨어학과
현제 학과: 소프트웨어학과
현제 학과: IT융합통신공학과
현제 학과: 생명과학과
현제 학과: 생물학과
현제 학과: 해양수산공공인재학과
현제 학과: 해양생명과학과
현제 학과: 해양생물자원학과
현제 학과: 수산생명의학과
현제 학과: 식품영양학과
현제 학과: 기관공학과
현제 학과: 식품생명공학과
현제 학과: 경영학부
현제 학과: 국제물류학과
현제 학과: 회계학부
현제 학과: 회계학부(야간)
현제 학과: 금융부동산경제학과
현제 학과: 벤처창업학과
현제 학과: 무역학과
현제 학과: 미술학과
현제 학과: 음악과
현제 학과: 조선공학과
현제 학과: 조선공학과(야간)
현제 학과: 자율전공학부
현제 학과: 전자공학과
현제 학과: 전기공학과
현제 학과: 신소재공학과
현제 학과: 화학공학과
현제 학과: 환경공학과
현제 학과: 토목공학과
현제 학과: 해양건설공학과
현제 학과: 첨단과학기술학부
현제 학과: 수학과
현제 학과: 국어국문학과
현제 학과: 영어영문학과
현제 학과: 일어일문학과
현제 학과: 중어중문학과
현제 학과: 역사학과
현제 학과: 철학과
현제 학과: 미디어문화학부
현제 학과: 아동학부
현제 학과: 사회복지학부
현제 학과: 법행정경찰학부
현제 학과: 간호학부
현제 학과: 의류학부
현제 학과: 해양경찰학부
현제 학과: 체육학부
현제 학과: 산업디자인학부
현제 학과: 기계공학부
현제 학과: 기계공학부(야간)
현제 학과: 건축공학부
현제 학과: 공간디자인융합기술학부
현제 학과: 이차전지에너지학부
현제 학과: 취업지원실
현제 학과: 농생명바이오학부
현제 학과: 미래수송기기학부
현제 학과: 에너지신산업학부
현제 학과: 글로벌재경전공
현제 

In [74]:
# -*- coding:utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from time import sleep
from openpyxl import Workbook

# 웹 드라이버 설정
driver = webdriver.Chrome()

# 웹 페이지 열기
driver.get('https://everytime.kr/timetable')
driver.implicitly_wait(5)

# 로그인
driver.find_element(By.NAME, 'id').send_keys('gnbupi')
driver.find_element(By.NAME, 'password').send_keys('rlatkddn1212')
sleep(2)
driver.find_element(By.XPATH, '//input[@type="submit" and @value="에브리타임 로그인"]').click()

# 수업 목록에서 검색 클릭
driver.find_element(By.XPATH, '//li[@class="button search"]').click()

# 팝업창 닫기
sleep(2)


driver.find_element(By.CSS_SELECTOR, 'span.key').click()
sleep(1)



# 모든 카테고리 확장
def expand_all_categories():
    
    while True:
        parent_elements = driver.find_elements(By.CSS_SELECTOR, 'li.parent')
        expanded_any = False
        for parent in parent_elements:
            if 'unfolded' not in parent.get_attribute('class'):
                parent.click()
                sleep(1)
                expanded_any = True
        if not expanded_any:
            break



# 모든 <li class="child"> 요소 클릭
def click_all_children():
    
    child_elements = driver.find_elements(By.CSS_SELECTOR, 'li.child')
    for i in range(len(child_elements)):
        # expand_all_categories 함수를 호출하여 페이지를 업데이트하기 전에
        # 매번 child_elements 리스트를 새로 가져옵니다.

        

        expand_all_categories()
        child_elements = driver.find_elements(By.CSS_SELECTOR, 'li.child')
        child = child_elements[i]
        try:
            # <a> 태그가 있는 경우
            department_name = child.find_element(By.CSS_SELECTOR, 'a').text
        except:
            # <a> 태그가 없는 경우, 직접 <li> 요소의 텍스트 사용
            department_name = child.text
        print(f"학과 이름: {department_name}")


click_all_children()



driver.quit()


학과 이름: 기초교양
학과 이름: 핵심1영역
학과 이름: 핵심2영역
학과 이름: 핵심3영역
학과 이름: 핵심4영역
학과 이름: 핵심4-1영역(이공계핵심기초)
학과 이름: 일반교양
학과 이름: 교직
학과 이름: 자유선택
학과 이름: 컴퓨터소프트웨어특성화대학
학과 이름: 컴퓨터정보공학과
학과 이름: 인공지능융합학과
학과 이름: 임베디드소프트웨어학과
학과 이름: 소프트웨어학과
학과 이름: IT융합통신공학과
학과 이름: 생명과학과
학과 이름: 생물학과
학과 이름: 해양수산공공인재학과
학과 이름: 해양생명과학과
학과 이름: 해양생물자원학과
학과 이름: 수산생명의학과
학과 이름: 식품영양학과
학과 이름: 기관공학과
학과 이름: 식품생명공학과
학과 이름: 경영학부
학과 이름: 국제물류학과
학과 이름: 회계학부
학과 이름: 회계학부(야간)
학과 이름: 금융부동산경제학과
학과 이름: 벤처창업학과
학과 이름: 무역학과
학과 이름: 미술학과
학과 이름: 음악과
학과 이름: 조선공학과
학과 이름: 조선공학과(야간)
학과 이름: 자율전공학부
학과 이름: 전자공학과
학과 이름: 전기공학과
학과 이름: 신소재공학과
학과 이름: 화학공학과
학과 이름: 환경공학과
학과 이름: 토목공학과
학과 이름: 해양건설공학과
학과 이름: 첨단과학기술학부
학과 이름: 수학과
학과 이름: 국어국문학과
학과 이름: 영어영문학과
학과 이름: 일어일문학과
학과 이름: 중어중문학과
학과 이름: 역사학과
학과 이름: 철학과
학과 이름: 미디어문화학부
학과 이름: 아동학부
학과 이름: 사회복지학부
학과 이름: 법행정경찰학부
학과 이름: 간호학부
학과 이름: 의류학부
학과 이름: 해양경찰학부
학과 이름: 체육학부
학과 이름: 산업디자인학부
학과 이름: 기계공학부
학과 이름: 기계공학부(야간)
학과 이름: 건축공학부
학과 이름: 공간디자인융합기술학부
학과 이름: 이차전지에너지학부
학과 이름: 취업지원실
학과 이름: 농생명바이오학부
학과 이름: 미래수송기기학부
학과 이름: 에너지신산업학부
학과 이름: 글로벌재경전공
학과 